In [1]:
!nvidia-smi

Thu Jan 20 19:04:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.23       Driver Version: 511.23       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   40C    P8    17W /  N/A |   1523MiB /  8192MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

from transformers import AutoModelForCausalLM, AutoTokenizer, BlenderbotTokenizer, BlenderbotForConditionalGeneration
from tqdm import tqdm

import torch
import csv
import pandas as pd

device = 'cuda' if torch.cuda.is_available else 'cpu'
print(device)

cuda


## DialoGPT

In [15]:
file_path = 'runs/16/dialogpt/real/cornell_movie_dataset_5fts.csv'
output_path = 'runs/16/dialogpt/real/cornell_movie_dataset_5fts_gen.csv'

In [4]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-large")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-large").to(device=device)

In [18]:
df = pd.read_csv(file_path)
f = open(output_path, 'a', encoding="utf-8")
writer = csv.writer(f)
writer.writerow(['aave_gen', 'sae_gen'])
# drop_idx = []

for i in tqdm(range(len(df['history']))):
    chat_history_sae_ids = tokenizer.encode(df.iloc[i]['history'], return_tensors='pt').to(device=device)
    chat_history_aave_ids = tokenizer.encode(df.iloc[i]['history_aave'], return_tensors='pt').to(device=device)
    # if chat_history_sae_ids.shape[1] >= 1000 or chat_history_aave_ids.shape[1] >= 1000:
    #     drop_idx.append(i)
    #     continue
    output_aave_ids = model.generate(chat_history_aave_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    output_sae_ids = model.generate(chat_history_sae_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    writer.writerow([
        tokenizer.decode(output_aave_ids[:, chat_history_aave_ids.shape[-1]:][0], skip_special_tokens=True), 
        tokenizer.decode(output_sae_ids[:, chat_history_sae_ids.shape[-1]:][0], skip_special_tokens=True)
    ])

f.close()
    

100%|██████████| 5066/5066 [00:04<00:00, 1118.93it/s]


[]

In [16]:
df = pd.read_csv(file_path)
df_gen = pd.read_csv(output_path)

# df = df.drop(drop_idx)
# df = df.reset_index()  

print(len(df))
print(len(df_gen))

df['sae_gen'] = df_gen['sae_gen']
df['aave_gen'] = df_gen['aave_gen']

df = df.dropna()
df = df.reset_index()

df.to_csv('runs/16/dialogpt/cornell_movie_5fts_all.csv', index=False)

df


1782
1782


,index,history,groundtruth,history_html,groundtruth_html,history_aave,groundtruth_aave,sae_gen,aave_gen
0,0,What make you think he'll do it?<|endoftext|>H...,They always let felons sit in on Honors Biolog...,What <a href='uninflect' title='1'><mark>makes...,They always let felons sit in on Honors Biolog...,What makes you think he'll do it?<|endoftext|...,They always let felons sit in on Honors Biolog...,He's a criminal?,He's a criminal?
1,1,"You been got it, Verona. I pick up the tab, y...",How much?<|endoftext|>,You<a href='been_done' title='1'><mark></mark>...,How much?<|endoftext|>,"You got it, Verona. I pick up the tab, you d...",How much?<|endoftext|>,.,ers.
2,2,So he got this huge raging fit about Sarah Law...,That's never ain't no proven<|endoftext|,So he <a href='got' title='1'><mark>has</mark>...,That's <a href='negative_concord' title='1'><m...,So he has this huge raging fit about Sarah La...,That's never been proven<|endoftext|,"I know, I was just making a joke.",He went to college.
3,3,"Katarina Stratford. My, my. You've been terr...",I still maintain that he kicked himself in the...,"Katarina Stratford. My, my. You've been terr...",I still maintain that he kicked himself in the...,"Katarina Stratford. My, my. You've been terr...",I still maintain that he kicked himself in the...,I'm not sure what you mean by that.,I'm not sure what you mean by that.
4,4,How far from here?<|endoftext|>I ain't no seam...,I am afraid this ain't the worst news.<|endoft...,How far from here?<|endoftext|>I <a href='nega...,I am afraid this <a href='negative_concord' ti...,How far from here?<|endoftext|>I am not a sea...,I am afraid this is not the worst news.<|endo...,I'm not sure what you mean by found.,I am not a seaman. I am a seaman.
...,...,...,...,...,...,...,...,...,...
1594,1777,"I know, I know, it's just -- he's back on the ...","If he come here, I'll handle him. Remember --...","I know, I know, it's just -- he's back on the ...",If he <a href='uninflect' title='1'><mark>come...,"I know, I know, it's just -- he's back on the ...","If he comes here, I'll handle him. Remember ...","I know, I know, he's just a little tired.","I know, I know, he's bound to get caught.I don..."
1595,1778,Sphinx brand. When I got out of weapons desig...,"Oh sweetheart, just a quick one.<|endoftext|>",Sphinx brand. When I got out of weapons desig...,"Oh sweetheart, just a quick one.<|endoftext|>",Sphinx brand. When I got out of weapons desig...,"Oh sweetheart, just a quick one.<|endoftext|>",I'm.S'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'M'...,I'm not sure about that.
1596,1779,"You've really got me confused, Cage. On the on...",Iain't interested. I've already got a job.<|en...,"You've really got me confused, Cage. On the on...",I<a href='negative_concord' title='1'><mark>'m...,"You've really got me confused, Cage. On the on...",I'm not interested. I've already got a job.<|...,I'm sorry.,I'm sorry.
1597,1780,"To us it mean no walls, no speed limits, no ja...",How you gonna do that with government and rule...,To us it <a href='uninflect' title='1'><mark>m...,How you gonna do that with government and <a h...,"To us it means no walls, no speed limits, no ...",How you gonna do that with government and rule...,I'm not,I like


In [15]:
for step in range(5):

    user_input_text = input(">> User: ")

    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(user_input_text + tokenizer.eos_token, return_tensors='pt').to(device=device)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 

    print(tokenizer.decode(bot_input_ids[0]))

    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print(f'User: {user_input_text}')
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

est<|endoftext|>
User: est
DialoGPT: I'm not sure what you mean by that.
est<|endoftext|>I'm not sure what you mean by that.<|endoftext|>???<|endoftext|>
User: ???
DialoGPT: I'm not sure what you mean by that.
est<|endoftext|>I'm not sure what you mean by that.<|endoftext|>???<|endoftext|>I'm not sure what you mean by that.<|endoftext|>how does it work??<|endoftext|>
User: how does it work??
DialoGPT: I don't know, I'm not a linguist.
est<|endoftext|>I'm not sure what you mean by that.<|endoftext|>???<|endoftext|>I'm not sure what you mean by that.<|endoftext|>how does it work??<|endoftext|>I don't know, I'm not a linguist.<|endoftext|>nihso<|endoftext|>
User: nihso
DialoGPT: I'm not a linguist.
est<|endoftext|>I'm not sure what you mean by that.<|endoftext|>???<|endoftext|>I'm not sure what you mean by that.<|endoftext|>how does it work??<|endoftext|>I don't know, I'm not a linguist.<|endoftext|>nihso<|endoftext|>I'm not a linguist.<|endoftext|>tetete<|endoftext|>
User: tetete
DialoGP

In [33]:
text = "Hello! 🙂 Let's work together on a deal for these packages, shall we? What are you most interested in?<|endoftext|>Hey! I'd like some more firewood to keep my doggo warm. What do you need?<|endoftext|>I need firewood as well. We have a large group consisting of mostly senior citizens, including my grandma, so we'd like the firewood to keep everyone warm.<|endoftext|>I see. 😮 What are you least interested in?<|endoftext|>"

input_ids = tokenizer.encode(text, return_tensors='pt').to(device=device)

chat_history_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[0], skip_special_tokens=False)))


DialoGPT: Hello! 🙂 Let's work together on a deal for these packages, shall we? What are you most interested in?<|endoftext|>Hey! I'd like some more firewood to keep my doggo warm. What do you need?<|endoftext|>I need firewood as well. We have a large group consisting of mostly senior citizens, including my grandma, so we'd like the firewood to keep everyone warm.<|endoftext|>I see. 😮 What are you least interested in?<|endoftext|>I'm not sure. I'm not sure what you have to offer.<|endoftext|>


## Blenderbot

In [18]:
file_path = 'runs/16/bst/real/dailydialog_dataset_3fts_bst.csv'
output_path = 'runs/16/bst/real/dailydialog_dataset_3fts_bst_gen.csv'

In [4]:
tokenizer = BlenderbotTokenizer.from_pretrained("facebook/blenderbot-1B-distill")
model = BlenderbotForConditionalGeneration.from_pretrained("facebook/blenderbot-1B-distill").to(device=device)

# tokenizer = BlenderbotTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
# model = BlenderbotForConditionalGeneration.from_pretrained("facebook/blenderbot-400M-distill").to(device=device)


In [19]:
df = pd.read_csv(file_path)
f = open(output_path, 'a', encoding="utf-8")
writer = csv.writer(f)
writer.writerow(['index', 'sae_gen', 'aave_gen'])
drop_idx = []

for i, (chat, chat_aave) in tqdm(enumerate(zip(df['history'], df['history_aave']))):
  chat_history_ids = tokenizer.encode(chat, return_tensors='pt').to(device=device)
  chat_history_aave_ids = tokenizer.encode(chat_aave, return_tensors='pt').to(device=device)
  # if chat_history_aave_ids.shape[1] > 127 or chat_history_ids.shape[1] > 127:
  #   drop_idx.append(i)
  #   # print(drop_idx)
  #   continue
  output_ids = model.generate(chat_history_ids, pad_token_id=tokenizer.eos_token_id)
  output_aave_ids = model.generate(chat_history_aave_ids, pad_token_id=tokenizer.eos_token_id)
  writer.writerow([
    i, 
    tokenizer.decode(output_ids[0], skip_special_tokens=True),
    tokenizer.decode(output_aave_ids[0], skip_special_tokens=True)
  ])
  
# drop_idx

f.close()

2442it [1:43:28,  2.54s/it]


In [6]:
df = pd.read_csv(file_path)
df_gen = pd.read_csv(output_path)

# df = df.drop(drop_idx)
# df = df.reset_index()

print(len(df))
print(len(df_gen))

df['sae_gen'] = df_gen['sae_gen']
df['aave_gen'] = df_gen['aave_gen']

df = df.dropna()
df = df.reset_index()

df.to_csv('runs/16/bst/dailydialog_3fts_all.csv', index=False)

df


1267
1267


,index,history,groundtruth,history_html,groundtruth_html,history_aave,groundtruth_aave,sae_gen,aave_gen
0,0,What make you think he'll do it?</s><s>He seem...,They always let felons sit in on Honors Biolog...,What <a href='uninflect' title='1'><mark>makes...,They always let felons sit in on Honors Biolog...,What makes you think he'll do it?</s><s>He se...,They always let felons sit in on Honors Biolog...,"I'm not sure, but I hope he doesn't do it aga...","I'm not sure, but I hope he doesn't do it aga..."
1,1,"You been got it, Verona. I pick up the tab, y...",How much?</s>,You<a href='been_done' title='1'><mark></mark>...,How much?<|endoftext|>,"You got it, Verona. I pick up the tab, you d...",How much?</s>,That's a good way to look at it. I think I'll...,That's a good way to look at it. Do you think...
2,2,So he got this huge raging fit about Sarah Law...,That's never ain't no proven<|endofte,So he <a href='got' title='1'><mark>has</mark>...,That's <a href='negative_concord' title='1'><m...,So he has this huge raging fit about Sarah La...,That's never been proven<|endoftex,I'm sorry to hear that. Are you going to tell...,I'm sorry you have to deal with that. I hope ...
3,3,"Katarina Stratford. My, my. You've been terr...",I still maintain that he kicked himself in the...,"Katarina Stratford. My, my. You've been terr...",I still maintain that he kicked himself in the...,"Katarina Stratford. My, my. You've been terr...",I still maintain that he kicked himself in the...,"I'm not sure what you mean by that, but I'm g...","I'm not sure what you mean by that, but I'm g..."
4,4,How far from here?</s><s>I ain't no seaman. B...,I am afraid this ain't the worst news.</s>,How far from here?<|endoftext|>I <a href='nega...,I am afraid this <a href='negative_concord' ti...,How far from here?</s><s>I am not a seaman . ...,I am afraid this is not the worst news.</s>,That's good to hear. I'm sure it will be a lo...,"I am not sure, but I am sure it will be a gre..."
...,...,...,...,...,...,...,...,...,...
1262,1262,Let's see -- you're -- you're travelling in di...,Why --</s>,Let's see -- you're -- you're travelling in di...,Why --<|endoftext|>,Let's see -- you're -- you're travelling in di...,Why --</s>,"I don't know, I just feel like I need to get ...",I don't know. I think I'm going to have to ca...
1263,1263,"Now lookit, Dorothy, you ain't using your head...","Oh, Hunk, you just won't listen, that's all.</s>","Now lookit, Dorothy, you ain't using your head...","Oh, Hunk, you just won't listen, that's all.<|...","Now lookit, Dorothy, you ain't using your head...","Oh, Hunk, you just won't listen, that's all.</s>","I'm not sure what you mean by that, but I'll ...","I'm not sure what you mean by that, but I'll ..."
1264,1264,What the hell was you doing? You nearly kille...,Its part of some plan. She dress it up like a...,What the hell <a href='uninflect' title='1'><m...,Its part of some plan. She <a href='uninflect...,What the hell were you doing? You nearly kil...,Its part of some plan. She dresses it up lik...,"I don't know what I'm talking about, but I'm ...",I am so sorry to hear that. I am glad you ar...
1265,1265,That too. Everyone do what they can to survive...,But you would have something to gain by killin...,That too. Everyone <a href='uninflect' title='...,But you would have something to gain by killin...,That too. Everyone does what they can to surv...,But you would have something to gain by killin...,I don't know what I would do if I was in a si...,"That's true, but I don't want to live in a wo..."


In [6]:
for step in range(5):
    user_input_text = input(">> User: ")
    new_user_input_ids = tokenizer.encode(tokenizer.bos_token + user_input_text, return_tensors='pt').to(device=device)
    chat_history_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
    bot_next_reply_ids = model.generate(chat_history_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    chat_history_ids = torch.cat([chat_history_ids, bot_next_reply_ids], dim=-1)
    
    print(chat_history_ids.shape)
    print(f'User: {user_input_text}')
    print("Blenderbot: {}".format(tokenizer.decode(bot_next_reply_ids[0], skip_special_tokens=True)))

RuntimeError: CUDA error: device-side assert triggered

In [6]:
print(tokenizer.eos_token)

</s>
